# Simulação da frota de taxis

$n \in \Bbb{N}$ taxis saem de casa.

Cada um faz $m \in \Bbb{N}$ corridas e vai pra casa.

Cada taxi começa a circular, encontra um passageiro, leva o passageiro, deixa o passageiro e volta a circular.

O tempo até achar um passageiro é $t \sim Exp(\lambda)$.

Para uma exibição mais limpa, considere $\lceil t\rceil$.

Mudanças de estados de taxis são registradas como eventos.

Os taxis 0, 1, 2, ... saem de suas casas nessa ordem e com 5 minutos de diferença.

## Importações

In [129]:
import random
import queue
from collections import namedtuple
from enum import Enum
from math import ceil

## Constantes

In [130]:
DEFAULT_NUMBER_OF_TAXIS = 5
DEFAULT_END_TIME = 180
SEARCH_DURATION = 3
TRIP_DURATION = 13
DEPARTURE_INTERVAL = 5

## Definição de evento e ações possíveis

In [131]:
Evento = namedtuple("Evento", "instante carro acao")

class Acao(Enum):
    SAIU = "saiu da garagem"
    PEGOU = "pegou um passageiro"
    DEIXOU = "deixou o passageiro"
    ACABOU = "indo pra casa descansar"

## Delegante

In [132]:
def taxi_processo(carro, corridas, tempo_inicial=0):
    tempo = yield Evento(tempo_inicial, carro, Acao.SAIU)
    for _ in range(corridas):
        tempo = yield Evento(tempo, carro, Acao.PEGOU)
        tempo = yield Evento(tempo, carro, Acao.DEIXOU)
    yield Evento(tempo, carro, Acao.ACABOU)

## Helper

In [133]:
def calcula_duracao(acao_anterior):
    """Calcula a duração da ação usando a distribuição exponencial"""
    if acao_anterior in [Acao.SAIU, Acao.DEIXOU]:
        interval = SEARCH_DURATION
    elif acao_anterior == Acao.PEGOU:
        interval = TRIP_DURATION
    elif acao_anterior == Acao.ACABOU:
        interval = 1
    else:
        raise ValueError(f"Valor inválido para 'acao_anterior': {acao_anterior}")
    return ceil(random.expovariate(1 / interval))

## Classe *Simulador*

In [134]:
class Simulador:
    def __init__(self, processos_list):
        self.eventos = queue.PriorityQueue()
        self.processos = processos_list

    def run(self, tempo_limite):
        for proc in self.processos:
            primeiro_evento = next(proc)
            self.eventos.put(primeiro_evento)

        instante_simulacao = 0
        while instante_simulacao < tempo_limite:
            if self.eventos.empty():
                print("*** fim dos eventos ***")
                break

            evento_atual = self.eventos.get()
            instante_simulacao, id_carro, acao_anterior = evento_atual
            print(f"taxi {id_carro}; t={instante_simulacao:3d}:", id_carro * "   ", evento_atual)
            tempo_seguinte = instante_simulacao + calcula_duracao(acao_anterior)
            try:
                if 0 <= id_carro < len(self.processos):
                    processo_ativo = self.processos[id_carro]
                    proximo_evento = processo_ativo.send(tempo_seguinte)
            except StopIteration:
                # del self.processos[id_carro]
                ...
            else:
                self.eventos.put(proximo_evento)
        else:
            msg = "*** Fim da execução da simulação: {} eventos pendentes ***"
            print(msg.format(self.eventos.qsize()))

## Rodando a simulação

In [135]:
def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS):
    taxis = [
        taxi_processo(i, (i + 1) * 2, i * DEPARTURE_INTERVAL)
        for i in range(num_taxis)
    ]

    sim = Simulador(taxis)
    sim.run(end_time)


main()

taxi 0; t=  0:  Evento(instante=0, carro=0, acao=<Acao.SAIU: 'saiu da garagem'>)
taxi 0; t=  4:  Evento(instante=4, carro=0, acao=<Acao.PEGOU: 'pegou um passageiro'>)
taxi 1; t=  5:     Evento(instante=5, carro=1, acao=<Acao.SAIU: 'saiu da garagem'>)
taxi 1; t=  6:     Evento(instante=6, carro=1, acao=<Acao.PEGOU: 'pegou um passageiro'>)
taxi 0; t=  8:  Evento(instante=8, carro=0, acao=<Acao.DEIXOU: 'deixou o passageiro'>)
taxi 1; t=  8:     Evento(instante=8, carro=1, acao=<Acao.DEIXOU: 'deixou o passageiro'>)
taxi 0; t=  9:  Evento(instante=9, carro=0, acao=<Acao.PEGOU: 'pegou um passageiro'>)
taxi 2; t= 10:        Evento(instante=10, carro=2, acao=<Acao.SAIU: 'saiu da garagem'>)
taxi 1; t= 11:     Evento(instante=11, carro=1, acao=<Acao.PEGOU: 'pegou um passageiro'>)
taxi 2; t= 11:        Evento(instante=11, carro=2, acao=<Acao.PEGOU: 'pegou um passageiro'>)
taxi 3; t= 15:           Evento(instante=15, carro=3, acao=<Acao.SAIU: 'saiu da garagem'>)
taxi 1; t= 17:     Evento(instante=